In [27]:
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By


In [28]:
from bs4 import BeautifulSoup

In [29]:
import bs4

In [30]:
import json

# I. Get a list of document titles

In [17]:
mypath = "edgedriver_win64\\msedgedriver.exe"
#s = Service(mypath)
driver = webdriver.Edge(executable_path= mypath)

In [18]:
url1 ="https://advance.lexis.com/search/?pdmfid=1516831&crid=f1531e5b-e2a5-448c-bdeb-34e123fd4485&pdsearchtype=SearchBox&pdtypeofsearch=searchboxclick&pdstartin=&pdsearchterms=legal+case&pdtimeline=Previous+2+Years+%7C+p2y&pdpsf=&pdquerytemplateid=&pdsf=&ecomp=3bJgkgk&prid=387b52f1-8a0e-4526-b00e-161688824ebf"
driver.get(url1)

In [19]:
#get title of documents

def get_doc_title(f):
    time.sleep(2)
    
    title_element = driver.find_element(By.XPATH,"//ol[@class='nexisuni-result-list']/li[{n}]/div/h2/a".format(n = f*2+2))
    
    title_text = title_element.text
    return title_text

In [20]:
for n in range(3):
    
    for k in range(250):
        for i in range(10):
            doc = driver.find_element(By.XPATH,"//ol[@class='nexisuni-result-list']/li[{n}]/div/div[2]/article/p".format(n = i*2+2))
            overview = doc.get_attribute("class")
            if overview == "overview min":
                title = get_doc_title(f = i)
                with open('document_title_{nth}_bward.txt'.format(nth = n),"a+", encoding="utf-8", newline="") as title_file:
                    title_file.write(title)
                    title_file.write('\n')
        time.sleep(2)
        next_page = driver.find_element(By.XPATH,"//a[@class='icon la-TriangleLeft action' and @href='#' and @data-action='prevpage']")
        next_page.click()
    


In [ ]:
driver.close()

# II. Get content of document from the above document title

In [21]:

with open("document_titles.txt","r") as infile:
    lines = infile.readlines()
    

In [22]:
document_titles = []

for line in lines:
    document_titles.append(line.strip("\n"))

In [23]:
unique_document_titles = list(set(document_titles[527:]))

In [31]:
def get_core_term():
    core_term_dict = {}
    core_terms = []
    all_elements = driver.find_element(By.XPATH,"//main[@role='main']")

    #all_elements = driver.find_element(By.XPATH,"//main[@id='4sk8k']")
    html_doc = all_elements.get_attribute('outerHTML')
    soup = BeautifulSoup(html_doc,"html.parser")
    child_lists = list(soup.select('.SS_LeftAlign')[0].children)
    for c in child_lists:
        if (c.text) == 'Core Terms':
            index_ = int(child_lists.index(c))
    coreterms = child_lists[index_+2]
    
    core_term_dict['Core Terms'] = coreterms
    return core_term_dict

In [32]:
def get_overview_doc():
    body_doc = driver.find_elements(By.XPATH,"//span[@class='SS_EditorialContent']/*")
    overview_dict = {}
    myKey = ''
    myValue = ''
    for i in range(len(body_doc)):
        tag_name = body_doc[i].tag_name
        if tag_name.startswith('span'):
            myKey = body_doc[i].text
        elif tag_name.startswith('p'):
            myValue = body_doc[i].text
        if myKey and myValue:
            overview_dict[myKey] = myValue
    return overview_dict


In [33]:
def get_body_doc():
    all_content_elements = driver.find_elements(By.XPATH,"//span[@class='SS_LeftAlign']/*")
    detail_dict = {}
    myKey = ''
    myValue = ''
    for i in range(len(all_content_elements)):
        attribute = all_content_elements[i].get_attribute('class')
        tag_name = all_content_elements[i].tag_name
        if tag_name.startswith('span') and attribute == 'SS_bf':
            if myKey:
                detail_dict[myKey] = myValue
                myKey = ''
                myValue = ''
            myKey = all_content_elements[i].text
        if tag_name.startswith('p') and myKey:
            myValue = myValue + all_content_elements[i].text
    detail_dict[myKey] = myValue
    return detail_dict

In [34]:
mypath = "edgedriver_win64\\msedgedriver.exe"
#s = Service(mypath)
driver = webdriver.Edge(executable_path= mypath)
url2 = 'https://advance-lexis-com.libproxy.library.unt.edu/search/?pdmfid=1516831&crid=cacd72d1-84b5-470a-8358-7b6f69c84643&pdsearchtype=SearchBox&pdtypeofsearch=searchboxclick&pdstartin=&pdsearchterms=Zive+v.+Sandberg%2C+610+S.W.3d+44&pdtimeline=Previous+2+Years+%7C+p2y&pdpsf=&pdquerytemplateid=&pdsf=&ecomp=bbJgkgk&prid=033a47fc-431f-4814-b8a3-ac5068c93b58'

In [36]:
driver.get(url2)

In [37]:
count = 0
for title in unique_document_titles[count:]:
    document_dict = {}
    core_term_dict = {}
    overview_dict = {}
    body_doc_dict = {}
    document_dict['Title'] = title
    #time.sleep(5)
    #driver.find_element(By.XPATH,"//textarea[@aria-label='Enter Search Term' and @type ='text']").clear()
    driver.find_element(By.TAG_NAME,'textarea').clear()
    time.sleep(3)
    driver.find_element(By.TAG_NAME,'textarea').send_keys(title)
    time.sleep(3)
    button = driver.find_element(By.XPATH,"//button[@type='button' and @class = 'icon la-Search mainSearch' and @aria-label='Search']")
    button.click()
    try:
        core_term_dict = get_core_term()
        time.sleep(4)
        overview_dict = get_overview_doc()
        time.sleep(4)
        body_doc_dict = get_body_doc()
        for i in core_term_dict:
            document_dict[i] = core_term_dict[i]
        for i in overview_dict:
            document_dict[i] = overview_dict[i]
        for i in body_doc_dict:
            document_dict[i] = body_doc_dict[i] 
    except:
        continue
    count =+ 1
    with open('200_legal_case_documents.json','a') as outfile:
        json.dump(document_dict,outfile)
        outfile.close()    
    if count % 10 == 0:
        print(count)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"textarea"}
  (Session info: MicrosoftEdge=108.0.1462.42)


In [103]:
driver.close()

In [116]:
f = open('legal_case_documents.json','r')
data = json.load(f)

In [119]:
f.close()